In [10]:
#| default_exp StackedAELatentDiffusionCond

# StackedAELatentDiffusionCond

> This is a version of one of Zach Evans' "stacked" conditioned diffusion-based autoencoders, for use with audio-algebra

**NOTE: For the 'Wrapper' version of this, see given_models.ipynb**

See Zach Evans' official [audio-diffusion](https://github.com/zqevans/audio-diffusion) for updates. 

Other parts of `audio-algebra` may use Scott Hawley's "Python Packaging Fork" of this: [https://github.com/drscotthawley/audio-diffusion](https://github.com/drscotthawley/audio-diffusion).  That fork is also "out of date" w.r.t. @zqevans' research; we'll sync back up someday!  See "LICENSE(S)" at the bottom of the notebook. 

In [11]:
#| hide
from nbdev.showdoc import *

In [23]:
#|export

# old imports from DVAEWrapper
from copy import deepcopy
import math
import os, sys
import subprocess
from collections import namedtuple
import numpy as np 
from contextlib import contextmanager
from pathlib import Path
from torch.nn.parameter import Parameter
import torch

import torchaudio
from torch import optim, nn, Tensor
from torch import multiprocessing as mp
from torch.nn import functional as F
import pytorch_lightning as pl
from tqdm import trange 

from einops import rearrange
from nwt_pytorch import Memcodes

# audio-diffusion imports
from diffusion.pqmf import CachedPQMF as PQMF # may require some manual labor/ symlinking directories
from encoders.encoders import AttnResEncoder1D
#from autoencoders.soundstream import SoundStreamXLEncoder
#from dvae.residual_memcodes import ResidualMemcodes
from decoders.diffusion_decoder import DiffusionAttnUnet1D
from diffusion.model import ema_update

from autoencoders.models import AudioAutoencoder
from audio_encoders_pytorch import Encoder1d
from ema_pytorch import EMA
from audio_diffusion_pytorch.modules import UNetCFG1d
from audio_diffusion_pytorch import NumberEmbedder  # pip install audio-diffusion-pytorch==0.0.97
#from audio_diffusion_pytorch.modules import UNetCFG1d

import laion_clap 
from laion_clap.training.data import get_audio_features


In [21]:
# not exported because only used for testing
from aeiou.core import get_device

In [13]:
#| export
# Define the noise schedule and sampling loop
def get_alphas_sigmas(t):
    """Returns the scaling factors for the clean image (alpha) and for the
    noise (sigma), given a timestep."""
    return torch.cos(t * math.pi / 2), torch.sin(t * math.pi / 2)

def alpha_sigma_to_t(alpha, sigma):
    """Returns a timestep, given the scaling factors for the clean image and for
    the noise."""
    return torch.atan2(sigma, alpha) / math.pi * 2

#def get_crash_schedule(t):
#    sigma = torch.sin(t * math.pi / 2) ** 2
#    alpha = (1 - sigma ** 2) ** 0.5
#    return alpha_sigma_to_t(alpha, sigma)

#from diffusion.utils import get_spliced_ddpm_cosine_schedul


In [14]:
#| export
@torch.no_grad()
def sample(model, x, steps, eta, step_list=None, **extra_args):
    """Draws samples from a model given starting noise."""
    print("sample: x.shape =",x.shape)
    ts = x.new_ones([x.shape[0]])
    if step_list is None:        # Create the noise schedule
        t = torch.linspace(1, 0, steps + 1)[:-1]
    else:                        # Noise schedule already created & passed in 
        t = step_list
    #print("sample: t = ",t)
        
    alphas, sigmas = get_alphas_sigmas(t)
    steps = len(t)  # just in case len(step_list) didn't match up with steps

    # The sampling loop
    for i in trange(steps):

        # Get the model output (v, the predicted velocity)
        #with torch.cuda.amp.autocast():
        v = model(x, ts * t[i], **extra_args).float()

        # Predict the noise and the denoised image
        pred = x * alphas[i] - v * sigmas[i]
        eps = x * sigmas[i] + v * alphas[i]

        # If we are not on the last timestep, compute the noisy image for the
        # next timestep.
        if i < steps - 1:
            # If eta > 0, adjust the scaling factor for the predicted noise
            # downward according to the amount of additional noise to add
            ddim_sigma = eta * (sigmas[i + 1]**2 / sigmas[i]**2).sqrt() * \
                (1 - alphas[i]**2 / alphas[i + 1]**2).sqrt()
            adjusted_sigma = (sigmas[i + 1]**2 - ddim_sigma**2).sqrt()

            # Recombine the predicted noise and predicted denoised image in the
            # correct proportions for the next step
            x = pred * alphas[i + 1] + eps * adjusted_sigma

            # Add the correct amount of fresh noise
            if eta:
                x += torch.randn_like(x) * ddim_sigma

    # If we are on the last timestep, output the denoised image
    #print("sample: pred.shape =",pred.shape)
    return pred





@torch.no_grad()
def resample(model_fn, 
             audio,  # note this is not real audio but rather latents of init audio
             steps=100, 
             eta=0, 
             sampler_type="v-ddim", 
             noise_level=1.0, 
             device='cuda',
             debug=True,
             batch_size=1,
             effective_length=2048, # TODO: make more flexible later hard-coded now for 22s runs. 
             **extra_args):
    """from Dance_Diffusion.ipynb: Noise the input"""
    if debug: print(f"resample: audio.shape = {audio.shape}, steps = {steps}, eta = {eta}, noise_level = {noise_level}")
    while len(audio.shape) < 3: 
        audio = audio.unsqueeze(0)  # add a batch dim and/or channel dim if needed
    if debug: print(f"resample: audio.shape = {audio.shape}")
    
    batch_size=1
    
    if sampler_type.startswith("v-"):
        t = torch.linspace(0, 1, steps + 1, device=device)
        step_list = t  # get_spliced_ddpm_cosine_schedule(t) #  get_crash_schedule(t)
        step_list = step_list[step_list < noise_level]
        alpha, sigma = get_alphas_sigmas(step_list[-1])
        noised = torch.randn(audio.shape, device=device)

        noised = audio.to(device) * alpha + noised * sigma
        noise = noised
        return sample(model_fn, noise, steps, eta, step_list=step_list.flip(0)[:-1], **extra_args)
    else:
        raise ValueError(f"Sorry, sampler {sampler_type} not implemented yet") 
        return None
  

In [27]:
#|export
class LatentAudioDiffusionAutoencoder(pl.LightningModule):
    def __init__(self, autoencoder: AudioAutoencoder):
        super().__init__()

        self.latent_dim = autoencoder.latent_dim

        self.second_stage_latent_dim = 32

        factors = [2, 2, 2, 2]

        self.latent_downsampling_ratio = np.prod(factors)

        self.downsampling_ratio = autoencoder.downsampling_ratio * self.latent_downsampling_ratio

        self.latent_encoder = Encoder1d(
            in_channels=self.latent_dim,
            out_channels = self.second_stage_latent_dim,
            channels = 128,
            multipliers = [1, 2, 4, 8, 8],
            factors =  factors,
            num_blocks = [8, 8, 8, 8],
        )

        self.latent_encoder_ema = deepcopy(self.latent_encoder)

        self.diffusion = DiffusionAttnUnet1D(
            io_channels=self.latent_dim,
            cond_dim = self.second_stage_latent_dim,
            n_attn_layers=0,
            c_mults=[512] * 10,
            depth=10
        )

        self.diffusion_ema = deepcopy(self.diffusion)

        self.diffusion_ema.requires_grad_(False)
        self.latent_encoder_ema.requires_grad_(False)

        self.autoencoder = autoencoder

        self.autoencoder.requires_grad_(False).eval()

        self.rng = torch.quasirandom.SobolEngine(1, scramble=True)

    def encode(self, reals):
        first_stage_latents = self.autoencoder.encode(reals)

        second_stage_latents = self.latent_encoder(first_stage_latents)

        second_stage_latents = torch.tanh(second_stage_latents)

        return second_stage_latents

    def decode(self, latents, steps=100, device="cuda", init_audio=None, init_strength=0.4, debug=True):
        first_stage_latent_noise = torch.randn([latents.shape[0], self.latent_dim, latents.shape[2]*self.latent_downsampling_ratio]).to(device)
        if init_audio is None: 
            first_stage_sampled = sample(self.diffusion, first_stage_latent_noise, steps, 0, cond=latents)
        else:
            first_stage_sampled = resample(self.diffusion, init_audio, steps, 0, cond=latents, noise_level=(1.0-init_strength))
        first_stage_sampled = first_stage_sampled.clamp(-1, 1)
        if debug: print("LatentAudioDiffusionAutoencoder: calling self.autoencoder.decode")
        decoded = self.autoencoder.decode(first_stage_sampled)
        if debug: print("LatentAudioDiffusionAutoencoder: decoded.shape =",decoded.shape)
        return decoded

    def load_ema_weights(self, ema_state_dict):
        own_state = self.state_dict()
        for name, param in ema_state_dict.items():
            if name.startswith("latent_encoder_ema."):
                new_name = name.replace("latent_encoder_ema.", "latent_encoder.")
                if isinstance(param, Parameter):
                    # backwards compatibility for serialized parameters
                    param = param.data
                own_state[new_name].copy_(param)
            if name.startswith("diffusion_ema."):
                new_name = name.replace("diffusion_ema.", "diffusion.")
                if isinstance(param, Parameter):
                    # backwards compatibility for serialized parameters
                    param = param.data
                own_state[new_name].copy_(param)


In [28]:
#|export
class StackedAELatentDiffusionCond(pl.LightningModule):
    def __init__(self, latent_ae: LatentAudioDiffusionAutoencoder, clap_module: laion_clap.CLAP_Module):
        super().__init__()

        self.latent_dim = latent_ae.second_stage_latent_dim
        self.downsampling_ratio = latent_ae.downsampling_ratio

        embedding_max_len = 1

        self.embedder = clap_module

        self.embedding_features = 512

       # self.timestamp_embedder = NumberEmbedder(features=self.embedding_features)

        '''# for samples:
        self.diffusion = UNetCFG1d(
            in_channels = self.latent_dim, 
            context_embedding_features = self.embedding_features,
            context_embedding_max_length = embedding_max_len, # + 2, #2 for timestep embeds
            channels = 256,
            resnet_groups = 8,
            kernel_multiplier_downsample = 2,
            multipliers = [2, 3, 4, 5],
            factors = [1, 2, 4],
            num_blocks = [3, 3, 3],
            attentions = [1, 1, 1, 1],
            attention_heads = 16,
            attention_features = 64,
            attention_multiplier = 4,
            attention_use_rel_pos=True,
            attention_rel_pos_max_distance=512,
            attention_rel_pos_num_buckets=128,
            use_nearest_upsample = False,
            use_skip_scale = True,
            use_context_time = True,
        )'''
        #for songs:
        self.diffusion = UNetCFG1d(
            in_channels = self.latent_dim, 
            context_embedding_features = self.embedding_features,
            context_embedding_max_length = embedding_max_len, # + 2, #2 for timestep embeds
            channels = 256,
            resnet_groups = 8,
            kernel_multiplier_downsample = 2,
            multipliers = [2, 3, 4, 4, 4, 4],
            factors = [1, 2, 2, 4, 4],
            num_blocks = [3, 3, 3, 3, 3],
            attentions = [0, 0, 2, 2, 2, 2],
            attention_heads = 16,
            attention_features = 64,
            attention_multiplier = 4,
            attention_use_rel_pos=True,
            attention_rel_pos_max_distance=2048,
            attention_rel_pos_num_buckets=256,
            use_nearest_upsample = False,
            use_skip_scale = True,
            use_context_time = True,
        )

        self.diffusion_ema = EMA(
            self.diffusion,
            beta = 0.9999,
            power=3/4,
            update_every = 1,
            update_after_step = 1
        )

        self.autoencoder = latent_ae

        self.autoencoder.requires_grad_(False)

        self.rng = torch.quasirandom.SobolEngine(1, scramble=True)

    def encode(self, reals):
        while len(reals.shape) < 3: reals = reals.unsqueeze(0)  # add batch and/or channel dims
        return self.autoencoder.encode(reals)

    def decode(self, latents, steps=100, init_audio=None, init_strength=0.4):
        return self.autoencoder.decode(latents, steps, device=self.device, init_audio=init_audio, init_strength=init_strength)

    def configure_optimizers(self):
        #optimizer = optim.Adam([*self.diffusion.parameters(), *self.timestamp_embedder.parameters()], lr=4e-5)
        optimizer = optim.Adam([*self.diffusion.parameters()], lr=4e-5)

        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=500, eta_min=1e-6)

        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        reals, jsons, timestamps = batch
        reals = reals[0]

        #timestamps = [[timestamp[0].item(), timestamp[1].item()] for timestamp in timestamps]
        mono_reals = reals.mean(dim=1)

        #with torch.cuda.amp.autocast():
        #timestamp_embeddings = self.timestamp_embedder(timestamps)

        with torch.no_grad():
            latents = self.encode(reals)
            audio_embeddings = self.embedder.get_audio_embedding_from_data(mono_reals, use_tensor=True)
            audio_embeddings = audio_embeddings.unsqueeze(1).to(self.device)
            #print(audio_embeddings)
            #print(audio_embeddings.shape)

        embeddings = audio_embeddings #torch.cat([text_embeddings, timestamp_embeddings], dim=1)

        # Draw uniformly distributed continuous timesteps
        t = self.rng.draw(reals.shape[0])[:, 0].to(self.device)

        # Calculate the noise schedule parameters for those timesteps
        alphas, sigmas = get_alphas_sigmas(t)

        # Combine the ground truth images and the noise
        alphas = alphas[:, None, None]
        sigmas = sigmas[:, None, None]
        noise = torch.randn_like(latents)
        noised_latents = latents * alphas + noise * sigmas
        targets = noise * alphas - latents * sigmas

        #with torch.cuda.amp.autocast():
        # 0.1 CFG dropout
        v = self.diffusion(noised_latents, t, embedding=embeddings, embedding_mask_proba = 0.1)
        mse_loss = F.mse_loss(v, targets)
        loss = mse_loss

        log_dict = {
            'train/loss': loss.detach(),
            'train/mse_loss': mse_loss.detach(),
            'train/lr': self.lr_schedulers().get_last_lr()[0],
            'train/ema_decay': self.diffusion_ema.get_current_decay()
        }

        self.log_dict(log_dict, prog_bar=True, on_step=True)
        return loss

    def on_before_zero_grad(self, *args, **kwargs):
        self.diffusion_ema.update()


Testing that:

In [29]:
first_stage_config = {"capacity": 64, "c_mults": [2, 4, 8, 16, 32], "strides": [2, 2, 2, 2, 2], "latent_dim": 32}
first_stage_autoencoder = AudioAutoencoder(**first_stage_config).eval()

latent_diffae = LatentAudioDiffusionAutoencoder(first_stage_autoencoder).eval()

clap_fusion, clap_amodel = True, "HTSAT-base"
device= get_device()
clap_module = laion_clap.CLAP_Module(enable_fusion=clap_fusion, device=device, amodel=clap_amodel).requires_grad_(False).eval()

ldc_model = StackedAELatentDiffusionCond(latent_ae=latent_diffae, clap_module=clap_module)
print("Success!") 

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Success!


# LICENSE(S)

In [8]:
#|export
'''
Besides the main LICENSE for this library overall, this particular file uses code by 
Zach Evans, who used some of Phil Wang's codes.  The licenses for those are as follows:

MIT License

Copyright (c) 2022,2023 Zach Evans

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.


MIT License

Copyright (c) 2022 Phil Wang

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
'''
''

''

In [30]:
#| hide
import nbdev; nbdev.nbdev_export()